In [61]:
import os
import numpy as np
import torch

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

from pathlib import Path
home = str(Path.home())

In [71]:
def predictions(clf, X_test, scaler=None):
    if not scaler == None:
        X_test   = scaler.transform(X_test)
    y_hat    = clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    return y_hat, y_hat_pr

def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    kwargs = {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 100, 'C': 0.01}
    clf = LogisticRegression(n_jobs=-1, **kwargs)
    clf.fit(X_train, y_train)
    y_hat, y_hat_pr = predictions(clf, X_test, scaler)
    
    return clf, scaler, y_hat, y_hat_pr

def RF(X_train, y_train, X_test, y_test):
    kwargs = {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 'log2', 'max_depth': 60, 'bootstrap': False}  
    clf = RandomForestClassifier(n_jobs=-1, **kwargs)
    clf.fit(X_train, y_train)
    y_hat, y_hat_pr = predictions(clf, X_test)
    
    return clf, y_hat, y_hat_pr

In [65]:
def prepare_datasets(nor, adv, factor=0.8):
    if len(nor.shape) > 2: 
        nor = nor.reshape((nor.shape[0], -1))
        adv = adv.reshape((adv.shape[0], -1))

    y_nor = np.zeros(nor.shape[0]).astype('int')
    y_adv = np.ones(adv.shape[0]).astype('int')

    x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

    X_train = np.concatenate((x_train_n, x_train_a))
    y_train = np.concatenate((y_train_n, y_train_a))

    X_test = np.concatenate((x_test_n, x_test_a))
    y_test = np.concatenate((y_test_n, y_test_a))

    return X_train, y_train, X_test, y_test

In [66]:
factor = 0.8
nr_mean = 3
mean_tables = []
nr_samples=2000

attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
detectors = ['lid', 'multilid']

results = {}

for it in range(nr_mean):
    # it = 2
    random_state = [21, 30, 65][it] # random.randint(100)
    final_table = np.zeros((len(attacks), 2))
    base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
    results[it] = {}

    for dataset in datasets:
        if dataset == 'imagenet':
            k=30
        else:
            k=20

        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue

            if not dataset in results[it]:
                results[it][dataset] = {}
            if not model in results[it][dataset]:
                results[it][dataset][model] = {}

            for detector in detectors:
                nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_normalos.pt"))[:nr_samples]
                adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_adverlos.pt"))[:nr_samples]
                nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_normalos.pt"))[:nr_samples]
                adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_adverlos.pt"))[:nr_samples]
                
                assert(nor_fgsm.shape[0] == nr_samples)
                assert(nor_bim.shape[0] == nr_samples)
                assert(nor_pgd.shape[0] == nr_samples)
                assert(nor_aa.shape[0] == nr_samples)
                assert(nor_df.shape[0] == nr_samples)
                assert(nor_cw.shape[0] == nr_samples)

                nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw]
                adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw]

                for source in range(len(nor)):
                    X_train_src, y_train_src, X_test_src, y_test_src = prepare_datasets(nor[source], adv[source], factor)
                    clf_lr, scaler, y_hat_lr, y_hat_pr_lr = LR(X_train_src, y_train_src, X_test_src, y_test_src)
                    clf_rf, y_hat_rf, y_hat_pr_rf = RF(X_train_src, y_train_src, X_test_src, y_test_src)

                    for target in range(len(nor)):
                        if source == target:
                            continue

                        auc_lr = 0;
                        auc_rf = 0
                        f1_lr = 0
                        f1_rf = 0

                        unique_classes = np.unique(y_test_src)
                        if len(unique_classes) < 2:
                            print("ROC AUC score is not defined when there is only one class.")
                            print(f"it:{it}, {dataset}, {model}, {detector}, {source}, {target}")
                        else:
                            X_train_tar, y_train_tar, X_test_tar, y_test_tar = prepare_datasets(nor[target], adv[target], factor)
                            y_test_lr, y_hat_pr_lr = predictions(clf_lr, X_test_tar, scaler)
                            auc_lr = round(100*roc_auc_score(y_test_tar, y_hat_pr_lr), 1)
                            # f1_lr =  round(100*f1_score(y_test_tar, y_hat_lr), 1)
                            acc_lr =  round(100*accuracy_score(y_test_tar, y_test_lr), 1)

                            y_test_rf, y_hat_pr_rf = predictions(clf_rf, X_test_tar)
                            auc_rf = round(100*roc_auc_score(y_test_tar, y_hat_pr_rf), 1)
                            # f1_rf =  round(100*f1_score(y_test_tar, y_hat_rf), 1)
                            acc_rf =  round(100*accuracy_score(y_test_tar, y_test_rf), 1)
                            print(f"it:{it}, {dataset}, {model}, {detector}, {attacks[source]}, {attacks[target]}, {auc_lr}, {acc_lr}")
                            print(f"it:{it}, {dataset}, {model}, {detector}, {attacks[source]}, {attacks[target]}, {auc_rf}, {acc_rf}")

                        if not detector in results[it][dataset][model]:
                            results[it][dataset][model][detector] = {}
                        if not attacks[source] in results[it][dataset][model][detector]: 
                            results[it][dataset][model][detector][attacks[source]] = {}
                        if not attacks[target] in results[it][dataset][model][detector][attacks[source]]: 
                            results[it][dataset][model][detector][attacks[source]][attacks[target]] = {}

                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['auc_lr'] = auc_lr
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['auc_rf'] = auc_rf
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['acc_lr'] = acc_lr
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['acc_rf'] = acc_rf

/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


it:0, cifar10, wrn28-10, lid, fgsm, bim, 76.4, 64.1
it:0, cifar10, wrn28-10, lid, fgsm, bim, 80.5, 70.5
it:0, cifar10, wrn28-10, lid, fgsm, pgd, 78.3, 65.4
it:0, cifar10, wrn28-10, lid, fgsm, pgd, 82.0, 69.9
it:0, cifar10, wrn28-10, lid, fgsm, aa, 86.4, 76.9
it:0, cifar10, wrn28-10, lid, fgsm, aa, 90.2, 81.6
it:0, cifar10, wrn28-10, lid, fgsm, df, 78.8, 66.6
it:0, cifar10, wrn28-10, lid, fgsm, df, 79.3, 68.0
it:0, cifar10, wrn28-10, lid, fgsm, cw, 75.9, 63.7
it:0, cifar10, wrn28-10, lid, fgsm, cw, 76.8, 67.8


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


it:0, cifar10, wrn28-10, lid, bim, fgsm, 89.8, 81.6
it:0, cifar10, wrn28-10, lid, bim, fgsm, 85.5, 75.9
it:0, cifar10, wrn28-10, lid, bim, pgd, 86.0, 77.4
it:0, cifar10, wrn28-10, lid, bim, pgd, 89.5, 82.6
it:0, cifar10, wrn28-10, lid, bim, aa, 96.8, 89.1
it:0, cifar10, wrn28-10, lid, bim, aa, 96.3, 88.5
it:0, cifar10, wrn28-10, lid, bim, df, 89.5, 81.9
it:0, cifar10, wrn28-10, lid, bim, df, 80.7, 70.8
it:0, cifar10, wrn28-10, lid, bim, cw, 83.6, 74.9
it:0, cifar10, wrn28-10, lid, bim, cw, 75.3, 65.9


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


...

it:2, imagenet, wrn50-2, multilid, aa, fgsm, 55.7, 50.5
it:2, imagenet, wrn50-2, multilid, aa, fgsm, 52.3, 50.1
it:2, imagenet, wrn50-2, multilid, aa, bim, 91.8, 76.6
it:2, imagenet, wrn50-2, multilid, aa, bim, 85.7, 61.6
it:2, imagenet, wrn50-2, multilid, aa, pgd, 93.5, 75.6
it:2, imagenet, wrn50-2, multilid, aa, pgd, 90.7, 72.4
it:2, imagenet, wrn50-2, multilid, aa, df, 54.2, 50.6
it:2, imagenet, wrn50-2, multilid, aa, df, 52.0, 50.1
it:2, imagenet, wrn50-2, multilid, aa, cw, 52.6, 50.2
it:2, imagenet, wrn50-2, multilid, aa, cw, 51.3, 50.2


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


it:2, imagenet, wrn50-2, multilid, df, fgsm, 55.7, 54.2
it:2, imagenet, wrn50-2, multilid, df, fgsm, 53.5, 51.5
it:2, imagenet, wrn50-2, multilid, df, bim, 89.7, 70.0
it:2, imagenet, wrn50-2, multilid, df, bim, 72.0, 65.1
it:2, imagenet, wrn50-2, multilid, df, pgd, 89.1, 73.2
it:2, imagenet, wrn50-2, multilid, df, pgd, 73.1, 65.8
it:2, imagenet, wrn50-2, multilid, df, aa, 97.6, 71.1
it:2, imagenet, wrn50-2, multilid, df, aa, 54.0, 53.4
it:2, imagenet, wrn50-2, multilid, df, cw, 53.5, 52.2
it:2, imagenet, wrn50-2, multilid, df, cw, 53.1, 50.9


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


it:2, imagenet, wrn50-2, multilid, cw, fgsm, 58.6, 55.5
it:2, imagenet, wrn50-2, multilid, cw, fgsm, 54.2, 54.2
it:2, imagenet, wrn50-2, multilid, cw, bim, 87.1, 71.0
it:2, imagenet, wrn50-2, multilid, cw, bim, 73.7, 66.4
it:2, imagenet, wrn50-2, multilid, cw, pgd, 85.5, 74.4
it:2, imagenet, wrn50-2, multilid, cw, pgd, 76.0, 66.4
it:2, imagenet, wrn50-2, multilid, cw, aa, 94.3, 76.5
it:2, imagenet, wrn50-2, multilid, cw, aa, 78.1, 69.2
it:2, imagenet, wrn50-2, multilid, cw, df, 53.6, 52.6
it:2, imagenet, wrn50-2, multilid, cw, df, 51.6, 51.2


# LID

In [72]:
source_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# source_attacks = ['fgsm',  'bim']
target_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
datasets =  ['cifar10', 'cifar100', 'imagenet']

lid_list_mean = []

for it_mean in range(nr_mean):
    lid_list = []

    for dataset in datasets:
        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            for source_attack in source_attacks:
                tmp = np.zeros((len(target_attacks)-1, 2))
                iter_tar = 0
                
                for target_attack in target_attacks:
                    if target_attack == source_attack:
                        continue
                        
                    for it, val in enumerate(results[it_mean][dataset][model]['lid'][source_attack][target_attack].items()):
                        # print(val)
                        if 'auc_lr' in val[0]:
                            tmp[iter_tar, 0] = val[1]
                        if 'acc_lr' in val[0]:
                            tmp[iter_tar, 1] = val[1]
                    iter_tar += 1
                
                lid_list.append(tmp.copy())
    lid_list_mean.append(np.stack(lid_list))

lid_mean = np.mean(lid_list_mean, axis=0)
lid_var = np.var(lid_list_mean, axis=0)

it = 0

for i in range(lid_mean.shape[0]):
    
    attacks_saved = target_attacks.copy()
    
    if i % 5 == 0:
        # print(it)
        source = source_attacks[it]
        it = it + 1
    attacks_saved.remove(source)

    latex_table = ""
    latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{"+f"{attacks_saved[int((i) % 5)].upper()}"+"} & "
    for j in range(lid_mean.shape[1]): 
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        
    latex_table = latex_table[:-2] + "\\\\"
    
    if int((i+1) % 5)==0:
        
        latex_table += " \n\hline"
        latex_table += " \n"
        
        start = i+1-5
        end = i+1
        avg_mean = lid_mean[start:end].mean(axis=1).flatten()
        avg_var  = lid_var[start:end].mean(axis=1).flatten()
        latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{$\mu$}"
        for k in range(len(avg_mean)):
            latex_table += f" & ${avg_mean[k]:.1f} \\pm {avg_var[k]:.1f}$"
        latex_table = latex_table + "\\\\ \n\hline"
        
        latex_table += "\n\n"
    
    print(latex_table)

\textbf{FGSM} & \textbf{BIM} & $62.3 \pm 102.5$ & $55.7 \pm 35.5$ & $62.9 \pm 118.6$ & $57.3 \pm 33.1$ & $66.8 \pm 191.5$ & $62.3 \pm 107.5$ & $79.2 \pm 0.8$ & $64.4 \pm 3.9$ & $76.3 \pm 0.2$ & $62.1 \pm 1.2$ \\
\textbf{FGSM} & \textbf{PGD} & $78.3 \pm 65.7$ & $61.2 \pm 208.8$ & $86.9 \pm 0.4$ & $77.7 \pm 0.5$ & $95.6 \pm 1.0$ & $87.2 \pm 1.8$ & $66.1 \pm 273.0$ & $61.1 \pm 217.0$ & $60.6 \pm 265.3$ & $58.3 \pm 137.2$ \\
\textbf{FGSM} & \textbf{AA} & $78.9 \pm 68.6$ & $60.8 \pm 229.0$ & $86.9 \pm 0.3$ & $79.5 \pm 0.5$ & $96.5 \pm 0.2$ & $88.5 \pm 0.7$ & $66.6 \pm 321.8$ & $61.7 \pm 268.3$ & $62.0 \pm 319.2$ & $59.4 \pm 176.1$ \\
\textbf{FGSM} & \textbf{DF} & $79.4 \pm 45.2$ & $60.4 \pm 191.4$ & $86.4 \pm 0.0$ & $74.0 \pm 1.3$ & $87.4 \pm 2.2$ & $74.3 \pm 2.2$ & $66.5 \pm 284.8$ & $60.8 \pm 204.7$ & $61.7 \pm 280.9$ & $58.4 \pm 136.7$ \\
\textbf{FGSM} & \textbf{CW} & $87.8 \pm 0.9$ & $79.4 \pm 0.0$ & $67.5 \pm 70.9$ & $55.0 \pm 50.0$ & $67.2 \pm 99.5$ & $54.8 \pm 46.1$ & $72.5 \pm 227.6

In [73]:
source_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# source_attacks = ['fgsm',  'bim']
target_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
datasets =  ['cifar10', 'cifar100', 'imagenet']

lid_list_mean = []

for it_mean in range(nr_mean):
    lid_list = []

    for dataset in datasets:
        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            for source_attack in source_attacks:
                tmp = np.zeros((len(target_attacks)-1, 2))
                iter_tar = 0
                
                for target_attack in target_attacks:
                    if target_attack == source_attack:
                        continue
                        
                    for it, val in enumerate(results[it_mean][dataset][model]['lid'][source_attack][target_attack].items()):
                        # print(val)
                        if 'auc_rf' in val[0]:
                            tmp[iter_tar, 0] = val[1]
                        if 'acc_rf' in val[0]:
                            tmp[iter_tar, 1] = val[1]
                    iter_tar += 1
                
                lid_list.append(tmp.copy())
    lid_list_mean.append(np.stack(lid_list))

lid_mean = np.mean(lid_list_mean,axis=0)
lid_var = np.var(lid_list_mean,axis=0)

it = 0

for i in range(lid_mean.shape[0]):
    
    attacks_saved = target_attacks.copy()
    
    if i % 5 == 0:
        # print(it)
        source = source_attacks[it]
        it = it + 1
    attacks_saved.remove(source)

    latex_table = ""
    latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{"+f"{attacks_saved[int((i) % 5)].upper()}"+"} & "
    for j in range(lid_mean.shape[1]): 
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        
    latex_table = latex_table[:-2] + "\\\\"
    
    if int((i+1) % 5)==0:
        
        latex_table += " \n\hline"
        latex_table += " \n"
        
        start = i+1-5
        end = i+1
        avg_mean = lid_mean[start:end].mean(axis=1).flatten()
        avg_var  = lid_var[start:end].mean(axis=1).flatten()
        latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{$\mu$}"
        for k in range(len(avg_mean)):
            latex_table += f" & ${avg_mean[k]:.1f} \\pm {avg_var[k]:.1f}$"
        latex_table += "\\\\ \n\hline"
        latex_table += "\n"
    
    print(latex_table)

\textbf{FGSM} & \textbf{BIM} & $62.3 \pm 166.8$ & $57.9 \pm 79.1$ & $61.0 \pm 220.7$ & $55.7 \pm 100.7$ & $68.5 \pm 243.0$ & $62.4 \pm 198.1$ & $79.6 \pm 0.2$ & $66.5 \pm 2.0$ & $75.7 \pm 0.6$ & $64.9 \pm 7.3$ \\
\textbf{FGSM} & \textbf{PGD} & $54.4 \pm 498.7$ & $54.1 \pm 238.9$ & $90.0 \pm 1.1$ & $82.3 \pm 3.9$ & $94.8 \pm 1.4$ & $88.0 \pm 2.0$ & $64.4 \pm 133.6$ & $59.8 \pm 61.7$ & $59.1 \pm 130.7$ & $56.0 \pm 49.3$ \\
\textbf{FGSM} & \textbf{AA} & $55.6 \pm 535.7$ & $55.5 \pm 355.3$ & $87.3 \pm 0.1$ & $79.6 \pm 0.2$ & $95.6 \pm 0.9$ & $89.2 \pm 1.0$ & $66.3 \pm 216.2$ & $61.1 \pm 141.7$ & $61.1 \pm 224.0$ & $57.4 \pm 115.8$ \\
\textbf{FGSM} & \textbf{DF} & $58.3 \pm 499.3$ & $57.9 \pm 234.0$ & $86.3 \pm 1.4$ & $73.2 \pm 0.4$ & $88.7 \pm 5.8$ & $77.7 \pm 5.2$ & $69.3 \pm 199.5$ & $62.0 \pm 104.3$ & $62.8 \pm 248.7$ & $57.0 \pm 91.6$ \\
\textbf{FGSM} & \textbf{CW} & $86.5 \pm 0.9$ & $78.3 \pm 0.5$ & $62.5 \pm 102.5$ & $54.7 \pm 44.2$ & $66.6 \pm 100.0$ & $56.2 \pm 76.1$ & $71.5 \pm 27

# multiLID

In [69]:
source_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# source_attacks = ['fgsm',  'bim']
target_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
datasets =  ['cifar10', 'cifar100', 'imagenet']

lid_list_mean = []

for it_mean in range(nr_mean):
    lid_list = []

    for dataset in datasets:
        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            for source_attack in source_attacks:
                tmp = np.zeros((len(target_attacks)-1, 2))
                iter_tar = 0
                
                for target_attack in target_attacks:
                    if target_attack == source_attack:
                        continue
                        
                    for it, val in enumerate(results[it_mean][dataset][model]['multilid'][source_attack][target_attack].items()):
                        # print(val)
                        if 'auc_lr' in val[0]:
                            tmp[iter_tar, 0] = val[1]
                        if 'acc_lr' in val[0]:
                            tmp[iter_tar, 1] = val[1]
                    iter_tar += 1
                
                lid_list.append(tmp.copy())
    lid_list_mean.append(np.stack(lid_list))

lid_mean = np.mean(lid_list_mean,axis=0)
lid_var = np.var(lid_list_mean,axis=0)

it = 0

for i in range(lid_mean.shape[0]):
    
    attacks_saved = target_attacks.copy()
    
    if i % 5 == 0:
        # print(it)
        source = source_attacks[it]
        it = it + 1
    attacks_saved.remove(source)

    latex_table = ""
    latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{"+f"{attacks_saved[int((i) % 5)].upper()}"+"} & "
    for j in range(lid_mean.shape[1]): 
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        
    latex_table = latex_table[:-2] + "\\\\"
    
    if int((i+1) % 5)==0:
        
        latex_table += " \n\hline"
        latex_table += " \n"
        
        start = i+1-5
        end = i+1
        avg_mean = lid_mean[start:end].mean(axis=1).flatten()
        avg_var  = lid_var[start:end].mean(axis=1).flatten()
        latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{$\mu$}"
        for k in range(len(avg_mean)):
            latex_table += f" & ${avg_mean[k]:.1f} \\pm {avg_var[k]:.1f}$"
        latex_table = latex_table + "\\\\ \n\hline"
        
        latex_table += "\n\n"
    
    print(latex_table)

\textbf{FGSM} & \textbf{BIM} & $74.9 \pm 62.2$ & $59.5 \pm 164.0$ & $90.4 \pm 1.6$ & $76.5 \pm 33.9$ & $91.0 \pm 28.8$ & $76.1 \pm 249.5$ & $83.0 \pm 4.8$ & $71.7 \pm 10.6$ & $79.1 \pm 1.5$ & $68.0 \pm 5.0$ \\
\textbf{FGSM} & \textbf{PGD} & $42.4 \pm 1244.3$ & $61.3 \pm 292.9$ & $45.2 \pm 1322.9$ & $64.1 \pm 373.6$ & $69.2 \pm 1808.0$ & $78.7 \pm 410.9$ & $32.5 \pm 1507.4$ & $58.9 \pm 248.6$ & $30.3 \pm 1279.1$ & $55.3 \pm 197.6$ \\
\textbf{FGSM} & \textbf{AA} & $93.7 \pm 2.5$ & $86.0 \pm 4.5$ & $50.2 \pm 976.9$ & $63.3 \pm 334.6$ & $70.0 \pm 1672.5$ & $79.7 \pm 438.1$ & $88.0 \pm 0.5$ & $80.0 \pm 0.0$ & $83.7 \pm 4.9$ & $76.1 \pm 0.0$ \\
\textbf{FGSM} & \textbf{DF} & $69.1 \pm 1144.7$ & $72.9 \pm 286.9$ & $70.7 \pm 959.5$ & $71.3 \pm 219.2$ & $70.6 \pm 1203.8$ & $75.0 \pm 311.5$ & $61.5 \pm 1432.3$ & $68.9 \pm 222.8$ & $57.9 \pm 1367.7$ & $65.1 \pm 160.8$ \\
\textbf{FGSM} & \textbf{CW} & $87.0 \pm 1.9$ & $79.6 \pm 0.9$ & $57.0 \pm 587.2$ & $54.7 \pm 43.6$ & $87.1 \pm 4.5$ & $69.2 \pm 

In [74]:
source_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# source_attacks = ['fgsm',  'bim']
target_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
datasets =  ['cifar10', 'cifar100', 'imagenet']

lid_list_mean = []

for it_mean in range(nr_mean):
    lid_list = []

    for dataset in datasets:
        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            for source_attack in source_attacks:
                tmp = np.zeros((len(target_attacks)-1, 2))
                iter_tar = 0
                
                for target_attack in target_attacks:
                    if target_attack == source_attack:
                        continue
                        
                    for it, val in enumerate(results[it_mean][dataset][model]['multilid'][source_attack][target_attack].items()):
                        # print(val)
                        if 'auc_rf' in val[0]:
                            tmp[iter_tar, 0] = val[1]
                        if 'acc_rf' in val[0]:
                            tmp[iter_tar, 1] = val[1]
                    iter_tar += 1
                
                lid_list.append(tmp.copy())
    lid_list_mean.append(np.stack(lid_list))

lid_mean = np.mean(lid_list_mean,axis=0)
lid_var = np.var(lid_list_mean,axis=0)

it = 0

for i in range(lid_mean.shape[0]):
    
    attacks_saved = target_attacks.copy()
    
    if i % 5 == 0:
        # print(it)
        source = source_attacks[it]
        it = it + 1
    attacks_saved.remove(source)

    latex_table = ""
    latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{"+f"{attacks_saved[int((i) % 5)].upper()}"+"} & "
    for j in range(lid_mean.shape[1]): 
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.1f} \\pm {variance:.1f}$ & "
        
    latex_table = latex_table[:-2] + "\\\\"
    
    if int((i+1) % 5)==0:
        
        latex_table += " \n\hline"
        latex_table += " \n"
        
        start = i+1-5
        end = i+1
        avg_mean = lid_mean[start:end].mean(axis=1).flatten()
        avg_var  = lid_var[start:end].mean(axis=1).flatten()
        latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{$\mu$}"
        for k in range(len(avg_mean)):
            latex_table += f" & ${avg_mean[k]:.1f} \\pm {avg_var[k]:.1f}$"
        latex_table += "\\\\ \n\hline"
        latex_table += "\n"
    
    print(latex_table)

\textbf{FGSM} & \textbf{BIM} & $62.2 \pm 249.6$ & $56.4 \pm 131.6$ & $86.5 \pm 1.8$ & $70.8 \pm 23.6$ & $82.2 \pm 336.8$ & $76.6 \pm 257.7$ & $84.4 \pm 2.0$ & $72.6 \pm 1.4$ & $81.2 \pm 1.3$ & $68.5 \pm 4.4$ \\
\textbf{FGSM} & \textbf{PGD} & $60.6 \pm 321.0$ & $56.3 \pm 154.8$ & $68.8 \pm 389.8$ & $66.2 \pm 317.8$ & $83.3 \pm 347.0$ & $80.7 \pm 316.8$ & $63.6 \pm 124.9$ & $54.7 \pm 2.2$ & $58.6 \pm 130.2$ & $52.2 \pm 3.7$ \\
\textbf{FGSM} & \textbf{AA} & $84.8 \pm 0.1$ & $73.0 \pm 3.5$ & $69.5 \pm 301.0$ & $65.2 \pm 250.2$ & $85.5 \pm 362.8$ & $83.3 \pm 324.8$ & $84.2 \pm 1.5$ & $57.6 \pm 3.5$ & $84.8 \pm 0.4$ & $57.2 \pm 0.9$ \\
\textbf{FGSM} & \textbf{DF} & $78.0 \pm 263.0$ & $69.2 \pm 100.4$ & $77.7 \pm 116.9$ & $61.2 \pm 2.4$ & $80.1 \pm 289.1$ & $68.8 \pm 77.4$ & $80.5 \pm 288.1$ & $62.4 \pm 44.3$ & $78.7 \pm 385.7$ & $60.5 \pm 49.6$ \\
\textbf{FGSM} & \textbf{CW} & $86.6 \pm 1.2$ & $78.7 \pm 1.7$ & $61.3 \pm 190.5$ & $49.7 \pm 0.1$ & $84.0 \pm 5.4$ & $55.3 \pm 0.3$ & $79.9 \pm 39